In [1]:
from pyforest import *
import pandas as pd
from pathlib import Path
from functional import seq
from tqdm import tqdm

In [2]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

## 翻译

In [3]:
#导入需要的包
import pandas as pd
import numpy as np
import requests
import json
def youdao_translate(word):
    # 有道词典 api
    url = 'http://fanyi.youdao.com/translate?smartresult=dict&smartresult=rule&smartresult=ugc&sessionFrom=null'
    # 传输的参数，其中 i 为需要翻译的内容
    key = {
        'type': "AUTO",
        'i': word,
        "doctype": "json",
        "version": "2.1",
        "keyfrom": "fanyi.web",
        "ue": "UTF-8",
        "action": "FY_BY_CLICKBUTTON",
        "typoResult": "true"
    }
    # key 这个字典为发送给有道词典服务器的内容
    response = requests.post(url, data=key)
    # 判断服务器是否相应成功
    if response.status_code == 200:
        # 然后相应的结果
        return response.text
    else:
        print("有道词典调用失败")
        # 相应失败就返回空
        return None
 
 
def trans_text(word):
    list_trans = youdao_translate(word)
    result = json.loads(list_trans)
    result = result['translateResult'][0][0]['tgt']
    return result

#调用
trans_text('Hello')
#结果《The 21st century》

'你好'

In [4]:
# !export GOOGLE_APPLICATION_CREDENTIALS=/home/dario/workspace/liyuan_paper/zdclink2-14952da40bc3.json

In [5]:
# !python3.8 -m pip install -q googletrans==3.1.0a0

# from googletrans import Translator
# translator = Translator()
# def google_trans(text):
#     return translator.translate(text, dest="zh-cn").text

# google_trans("Hello")

## 数据

In [6]:
root_dir = Path("/home/dario")
data_file = root_dir / "workspace/causal_experiment/data/GSE42057_series_matrix.txt"

In [7]:
# df = pd.read_csv(data_file, delimiter="\t", skiprows=78, index_col=0, header=None)
df = pd.read_csv(data_file, delimiter="\t", skiprows=33, index_col=0, header=0)

/home/dario/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
df.dropna(inplace=True)

In [21]:
matrix_data = df.tail(-43)

In [35]:
(seq(df.index)
 .filter(lambda x: x.startswith("!"))
 .map(lambda x: trans_text(x[1:].replace("_", " ")))
)

['样品加入地理', '样品状态', '样品提交日期', '示例最后更新日期', '样本类型', '采样通道数', '示例源名称ch1', '样品生物ch1', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', 'ch1样本特征', '样品分子ch1', '样本提取协议ch1', '样品标签ch1', '样品标签协议ch1', '样本taxid ch1', 'hyb协议样本', '样品扫描协议', '样本数据处理', '样品平台id', '示例联系人姓名', '样品联系邮件', '样品联系实验室', '样品接触研究所', '样品联系地址', '样品联系市', '样品接触状态', '样品联系邮政编码', '样品接触的国家', '补充文件示例', '样本数据行数', '样的关系', '样的关系', '系列矩阵表开始', '系列矩阵表结束']

### 映射

In [22]:
tmp = !head -17 data/GPL570-55999.txt

In [23]:
map_df = pd.read_csv("data/GPL570-55999.txt", sep="\t", skiprows=16, index_col=0)

/home/dario/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
map_df["Process"] = (
    seq(map_df["Gene Ontology Biological Process"])
     .map(lambda x: x if isinstance(x, str) else "None//None//None")
     .map(lambda x: x.split("///"))
     .map(lambda x: [y.split("//")[1].strip() for y in x])
     .map(lambda x: ','.join(x))
    #  .map(lambda x: [y for y in x if not (y.startswith("inferred from") or y.startswith("traceable author statement"))])
).to_list()

In [25]:
map_df = map_df[["Gene Title", "Process"]]

In [26]:
map_df.fillna("None", inplace=True)

In [27]:
map_df.loc["117_at", "Gene Title"]
map_df.loc["117_at", "Process"]

'cell morphogenesis,ATP catabolic process,response to stress,response to unfolded protein,cellular response to heat,protein refolding,cellular heat acclimation'

In [28]:
token_2_title = dict(zip(map_df[["Gene Title"]].index, map_df["Gene Title"].values))

In [17]:
# import time
# with open("data/GSE42057_token_2_title.txt", "w") as f:
#     for token in tqdm(df.index):
#         try:
#             f.write(f"{token}\t{google_trans(token_2_title.get(token))}\n")
#         except Exception:
#             pass
#         time.sleep(2)

## 计算

In [52]:
import causal_discovery
causal_discovery.__version__

'1.0.5'

In [53]:
from causal_discovery.parameter.log import config_log
config_log(
    "causal_discovery",
    "local_ng_cd",
    log_root="/home/dario/workspace/liyuan_paper/log",
    print_terminal=False,
    enable_monitor=True,
)

In [54]:
from causal_discovery.algorithm.local_ng_cd.local_ng_cd import local_ng_cd
from causal_discovery.data_prepare.matrix_data import get_matrix_data
from causal_discovery.parameter.algo import DataType, LocalNgCdParam
from causal_discovery.parameter.env import select_xp
from causal_discovery.parameter.error import DataTypeError

xp = select_xp()

In [55]:
def get_causal_result(target):
    matrix_data = get_matrix_data(
        target,
        corr_filter=True,
        need_norm=False,
        ret_df=df.T,
    ).T

    # 暴露出来允许修改的参数
    param = LocalNgCdParam()
    param.candidate_two_step = False
    param.target_index = list(matrix_data.columns).index(target)
    # 变量token -> name
    index_map = dict(enumerate(matrix_data.columns))
    # 调用主函数计算因果
    edges_trust, synthesize_effect = local_ng_cd(
        xp.asarray(matrix_data.T.to_numpy()), param, synthesize=True
    )
    edges_trust = [
        (index_map.get(int(causal), "N"), index_map.get(int(result), "N"), weight)
        for causal, result, weight in edges_trust
    ]
    return edges_trust

In [19]:
target = "1294_at"
trans_text(token_2_title[target])

'microRNA 5193 / / / ubiquitin-like修改器激活酶7'

In [20]:
edges_trust = get_causal_result(target)
pd.DataFrame([(trans_text(token_2_title[x]), trans_text(token_2_title[y]), z) for x, y, z in edges_trust])

,0,1,2
0,排序nexin 6,microRNA 5193 / / / ubiquitin-like修改器激活酶7,10.222674
1,microRNA 5193 / / / ubiquitin-like修改器激活酶7,translin,-13.021033


In [57]:
trans_text(x_process)

'蛋白质磷酸化酶联受体蛋白信号通路,磷酸化,peptidyl-tyrosine磷酸化,cytokine-mediated信号通路、细胞内信号转导,interleukin-2-mediated信号通路,应对抗生素,蛋白质自身磷酸化,interferon-gamma-mediated信号通路,调节interferon-gamma-mediated信号通路,I型干扰素信号通路,调节I型interferon-mediated信号通路'

In [63]:
import sys

In [69]:
errors = []
with open("data/result.csv", "w") as f:
    f.write("target\ttoken_x\ttoken_y\ttitle_x\ttitle_y\tprocess_x\tprocess_y\tweight\n")
    for target in tqdm(df.index):
        try:
            target_name = trans_text(token_2_title[target])
            edges_trust = get_causal_result(target)
            for x, y, z in edges_trust:
                x_name = trans_text(token_2_title[x])
                y_name = trans_text(token_2_title[y])
                x_process = trans_text(map_df.loc[x, "Process"])
                y_process = trans_text(map_df.loc[y, "Process"])
                f.write(f"{target}\t{x}\t{y}\t{x_name}\t{y_name}\t{x_process}\t{y_process}\t{z}\n")
        except Exception as e:
            print(sys.exc_info())
            errors.append(target)

  0%|          | 0/12531 [00:12<?, ?it/s]


KeyboardInterrupt: 

### cdt

In [29]:
import random
import os
import numpy as np
import torch

import cdt
import networkx as nx
from pprint import pprint

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.


In [30]:
def seed_torch(seed=1024):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


seed_torch()

In [32]:
matrix_data = matrix_data.T

In [33]:
glasso = cdt.independence.graph.Glasso()
skeleton = glasso.predict(matrix_data)

MemoryError: Unable to allocate 1.17 GiB for an array with shape (12531, 12531) and data type float64